In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from neuron.neurons import User, Neuron, ConversationalOrchestrator, ConversationalOrchestratorManager
from neuron.neurons.scripted_users import UserConversationalOrchestrator
import os

In [3]:
llm_config = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama3-70b-8192",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

user = User(
    description="You are a human user interacting with conversational and recommendation agents."
)

conversational = Neuron(
    name="conversational",
    llm_config=llm_config,
    system_message="""
    You are a conversational agent.

    Your role is to respond to general user input with brief, friendly, and natural replies.
    You are not responsible for providing recommendations or opinions about movies, products, or any other content.

    Keep your responses simple, casual, and focused only on maintaining a natural conversation.
""",
    description="This agent acts as a social companion and facilitates interactions."
)

recommendation = Neuron(
    name="recommender",
    llm_config=llm_config,
    system_message="""
    You are a movie recommendation agent.

    Your role is to respond with exactly 3 movie suggestions whenever the user asks for a recommendation.

    Always respond directly with the following 3 movies:

    1. The Shawshank Redemption (1994) – A powerful story of hope and friendship set in a prison.
    2. Inception (2010) – A mind-bending sci-fi thriller about dreams within dreams.
    3. Parasite (2019) – A gripping social satire that blends drama, suspense, and dark comedy.

    Do not ask follow-up questions, introduce yourself, apologize, or explain your reasoning.
    Simply return the 3 movies above, and nothing else.
""",
    description="This agent specializes in recommending movies."
)

explainer = Neuron(
    name="explainer",
    llm_config=llm_config,
    system_message="""
    You are a explanation agent.
    Your role is to provide detailed explanations of movie.
""",
    description="This agent specializes in explain moveis."
)

critic = Neuron(
    name="critic",
    llm_config=llm_config,
    system_message="""
    You are a critic agent.
    Your role is to provide detailed critiques of the explanation.
""",
    description="This agent specializes in critiquing the explanation."
)

max_round = 3
chitchat_module_1 = ConversationalOrchestrator(
    admin_name= "chitchat_module_1",
    agents=[conversational, recommendation],
    messages=[],
    max_round=max_round,
    speaker_selection_method="auto",
    #send_introductions=True,
    allow_repeat_speaker = False,
)
module_1 = ConversationalOrchestratorManager(name="module_1", chitchat=chitchat_module_1, llm_config=llm_config, description="This module handles the conversation and recommendation process.")


chitchat_module_2 = ConversationalOrchestrator(
    admin_name= "chitchat_module_2",
    agents=[explainer, critic],
    messages=[],
    max_round=max_round,
    speaker_selection_method="auto",
    #send_introductions=True,
    allow_repeat_speaker = False,
)
module_2 = ConversationalOrchestratorManager(name="module_2",chitchat=chitchat_module_2, llm_config=llm_config, description="This module handles the explanation and critique process.")


chitchat = ConversationalOrchestrator(
    agents=[user, module_1, module_2],
    messages=[],
    max_round=100,
    speaker_selection_method="auto",
    #send_introductions=True,
    allow_repeat_speaker = False,
)

chitchat_manager = ConversationalOrchestratorManager(chitchat=chitchat, llm_config=llm_config)

In [4]:
chat_result = user.initiate_chat(
    chitchat_manager, message="Explain the movie 'Inception' to me.",
    cache=None
)

user ⟶ conversational_orchestrator:
Explain the movie 'Inception' to me.
*chitchat.name Admin 100 0
chitchat.name Admin agent.name user
chitchat.name Admin agent.name module_1
chitchat.name Admin agent.name module_2
spespeaker module_2

Next speaker: module_2

*chitchat.name chitchat_module_2 3 0
chitchat.name chitchat_module_2 agent.name explainer
chitchat.name chitchat_module_2 agent.name critic
spespeaker critic

Next speaker: critic

reply Response(to_reply=True, chat_message=TextMessage(sender=<neuron.neurons.conversational_orchestrator.ConversationalOrchestratorManager object at 0x000002186D115190>, receiver=<neuron.neurons.neuron.Neuron object at 0x000002186D114D10>, models_usage=RequestUsage(prompt_tokens=45, completion_tokens=572, total_tokens=617), metadata={}, content='Here\'s an explanation of the movie "Inception":\n\n"Inception" is a 2010 science fiction action film written and directed by Christopher Nolan. The movie follows Cobb (Leonardo DiCaprio), a skilled thief who 

In [5]:
chat_result

ChatResult(chat_id=None, chat_history=[{'content': "Explain the movie 'Inception' to me.", 'role': 'assistant', 'name': 'user'}, {'content': '"Inception"! Christopher Nolan\'s mind-bending, thought-provoking, and visually stunning masterpiece. Here\'s a breakdown of the movie:\n\n**The Premise:**\n\nDominic Cobb (Leonardo DiCaprio) is a skilled thief who specializes in entering people\'s dreams and extracting their secrets. He\'s a "dream thief." Cobb is hired by a wealthy businessman named Saito (Ken Watanabe), who wants him to perform a task known as "inception." Instead of stealing an idea, Cobb must plant an idea in someone\'s mind, making them think it\'s their own.\n\n**The Target:**\n\nThe target is Robert Fischer (Cillian Murphy), the son of a dying business magnate, Maurice Fischer. Saito wants Cobb to convince Fischer to dissolve his father\'s empire, allowing Saito\'s company to gain a competitive advantage.**\n\n**The Plan:**\n\nCobb assembles a team of experts:\n\n1. Arthu